## Refactor the baseline itemCF_numba, and add feature store function to it

In [1]:
import os
import gc
import heapq
import pickle
import numba as nb
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import math


In [2]:
%%time
df = pd.read_csv("../../allData/validationData/train_meta_data.csv")
df_test = pd.read_csv("../../allData/validationData/test_meta_data.csv")
df = pd.concat([df, df_test]).reset_index(drop = True)
npz = np.load("../../allData/validationData/train_core_data.npz")
npz_test = np.load("../../allData/validationData/test_core_data.npz")
aids = np.concatenate([npz['aids'], npz_test['aids']])
ts = np.concatenate([npz['ts'], npz_test['ts']])
ops = np.concatenate([npz['ops'], npz_test['ops']])

df["start_idx"] = df['total_action'].cumsum().shift(1).fillna(0).astype(int)
df["end_time"] = ts[df["start_idx"] + df["total_action"] - 1]

CPU times: user 6.75 s, sys: 2.96 s, total: 9.71 s
Wall time: 10.1 s


In [3]:
df.head()

,session,total_action,session_start_time,session_end_time,start_idx,end_time
0,0,147,1659304800,1661103727,0,1661103727
1,1,27,1659304800,1660857067,147,1660857067
2,2,13,1659304800,1660577379,174,1660577379
3,3,226,1659304800,1661109666,187,1661109666
4,4,3,1659304800,1659304900,413,1659304900


## 1. Training -- Derive ItemCF similarity Matrix

#### CONSTANTS

In [3]:
## Define constants
PARALLEL = 1024
LOOKBACK_WINDOW = 100   ## only fit the latest LOOKBACK_WINDOW to train the sim matrix
#TOPN = 20
ACTION_WEIGHTS = np.array([1.0, 6.0, 3.0])

#### Section A: Utils Functions 
1. Count Item Total likes: The similary score will be normalized by "Item Total Like Scores". In theory, popular items should have less weight in simiarity score.
2. Trimming function: Helpful managing memoery usage. 
3. Method for normalization: Mostly item total like normalization, and max norm(make all sim score between 0 and 1) of the score. 

In [4]:
# ==================================
# Methods for counting Item Total Likes
# ==================================
@nb.jit(nopython=True)
def getItemTotalLikesNaive(aids, ops, item_total_likes, action_weights):
    """
    Stores the total like score of itemXXX in item_total_likes, based on action_weights parameter. np.array([X, Y, Z])
    """
    for idx, item in enumerate(aids):
        if item not in item_total_likes: 
            item_total_likes[item] = 0
        item_total_likes[item] += action_weights[ops[idx]]   ## TODO: For time decay, consider replace with 1, for iuf keep this. 

# ==================================
# Methods for rank and trim the sim score dict
# ==================================
@nb.jit(nopython = True)
def heap_topk(item_cnt_dict, cap):
    """
    get the top cap(k) elements of the cnt dict based on value, using a min-heap structure
    """
    dic = nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)
    q = [(np.float64(0), np.int64(0)) for _ in range(0)]  ## generate empty queue to implement a heap, 
    for item_ref, sim_score in item_cnt_dict.items():   ## read in the dict in heap structure
        heapq.heappush(q, (sim_score, item_ref))   ## push the <sim_score, item_ref_id> pair into min-heap, using sim_score for order
        if len(q) > cap:
            heapq.heappop(q)
            
    res = [heapq.heappop(q) for _ in range(len(q))][::-1]
    for i in range(len(res)):
        dic[res[i][1]] = res[i][0]
    
    return dic
   
@nb.jit(nopython = True)
def trim_simMatrix_topk(fullSimMatrix, k = 50):
    """
    trim top k items of each "itemX: {itemY: score1, ...}" pair in fullSimMatrix based on sim scores. 
    """
    for item, item_cnt_dict in fullSimMatrix.items():
        fullSimMatrix[item] = heap_topk(item_cnt_dict, k)

# ==================================
# Methods for score normalization
# ==================================

# @nb.jit(nopython=True)
# def itemTotalLikeNorm(fullSimMatrix, item_total_likes):
#     for aid_1, relations in fullSimMatrix.items():
#         for aid_2, sim_score in relations.items():
#             fullSimMatrix[aid_1][aid_2] = sim_score / (item_total_likes[aid_1] * item_total_likes[aid_2]) ** 0.1  ## TODO: consider 0.1 or other small number
            
@nb.jit(nopython=True)
def maxNormSimMatrix(fullSimMatrix):
    for aid_1, relations in fullSimMatrix.items():
        max_num = -np.inf
        for _, sim_score in relations.items():
            if sim_score > max_num:
                max_num = sim_score
        ## DEGUG use, delete later
        if max_num == 0:
            print(aid_1)
            print(fullSimMatrix[aid_1])
        for aid_2, sim_score in relations.items():
#             if max_num == 0:
#                 max_num += 0.001
            fullSimMatrix[aid_1][aid_2] = sim_score / max_num

#### Section B: Sim Score Computation functions

In [5]:

@nb.jit(nopython=True)
def getSimScoresSingleRow(pairs_this_row, start_time, start_idx, length, aids, ts, ops, item_total_likes, action_weights, mode):
    """
    Get the sim scores of items within single session, can be ran in parallel within each batch. 
    """
    max_idx = start_idx + length
    min_idx = max(max_idx - LOOKBACK_WINDOW, start_idx)  
    for i in range(min_idx, max_idx):
        for j in range(i+1, max_idx):
            if ts[j] - ts[i] > 2 * 60 * 60: continue  #TODO: try 2h only
            if aids[i] == aids[j]: continue
            
            if mode == "cosine":
                w_ij = action_weights[ops[j]] 
                w_ji = action_weights[ops[i]] 
            elif mode == "iuf":  ## penalize users that had lots of actions TODO: consider location weight
                if ts[max_idx] - start_time > 24 * 60 * 60:
                    special_factor = 0.8
                else:
                    special_factor = 1.0
                
                loc_weight = 0.5**(abs(i-j))   #math.exp(-0.02 * abs(i-j)) 
                time_gap_weight = 0.5 ** (abs(ts[i]-ts[j]) / (1.5*60*60))  
                w_ij = special_factor * action_weights[ops[j]] * time_gap_weight * loc_weight / math.log1p(length)
                w_ji = special_factor * action_weights[ops[i]] * time_gap_weight * loc_weight / math.log1p(length)
            elif mode == "time_decay":
                ## calculate some time weights of each item, more weights are given when ts is later. #TODO: try adding (i-j) location weight, exponential weight, 0.5 ** (abs(i-j + 1)), 
                loc_weight = 0.5**(abs(i-j))   #math.exp(-0.02 * abs(i-j)) 
                #time_i = 1 + 0.1 ** ((1662328791-ts[i])/(1662328791-1659304800)) #1 + 3 * (ts[i] + start_time - 1659304800) / (1662328791 - 1659304800) #  #(1 - 0.8 *(TEST_END_TS - ts[i]) / TIME_SPAN) ** 0.5 # 0.2~1 #   ## time decay weight for item i 
                #time_j = 1 + 0.1 ** ((1662328791-ts[j])/(1662328791-1659304800))  # 1 + 3 * (ts[j] + start_time - 1659304800) / (1662328791 - 1659304800) # #  #(1 - 0.8 *(TEST_END_TS - ts[j]) / TIME_SPAN) ** 0.5   # 
                time_i = 1 + 1/(1 + math.exp(10*( ((1662328791-ts[i])/(1662328791-1659304800)) - 0.6  )))
                time_j = 1 + 1/(1 + math.exp(10*( ((1662328791-ts[j])/(1662328791-1659304800)) - 0.6  )))
                
                time_gap_weight = 0.5 ** (abs(ts[i]-ts[j]) / (1.5*60*60))  
                
                w_ij = action_weights[ops[j]] * loc_weight * time_gap_weight * time_i / math.log1p(length)
                w_ji = action_weights[ops[i]] * loc_weight * time_gap_weight * time_j / math.log1p(length)
                
            pairs_this_row[(aids[i], aids[j])] = w_ij / (item_total_likes[aids[i]] * item_total_likes[aids[j]]) ** 0.1
            pairs_this_row[(aids[j], aids[i])] = w_ji / (item_total_likes[aids[i]] * item_total_likes[aids[j]]) ** 0.1

@nb.jit(nopython=True, parallel=True, cache=True)
def getSimScoreBatch(aids, ts, ops, rows, fullSimMatrix, action_weights, item_total_likes, mode="cosine"):
    nrows = len(rows)
    pairs_this_batch = [{(0, 0): 0.0 for _ in range(0)} for _ in range(nrows)]
    ## get the sim scores of each batch in seperate sub dict in pairs_this_batch
    for row_i in nb.prange(nrows):  ## run each row of the batch in parallel
        _, start_idx, length, start_time = rows[row_i]
        getSimScoresSingleRow(pairs_this_batch[row_i], start_time, start_idx, length, aids, ts, ops, item_total_likes, action_weights, mode)
    ## merge pairs_this_batch into the fullSimMatrix
    for row_i in range(nrows):
        for (aid1, aid2), score in pairs_this_batch[row_i].items():
            if aid1 not in fullSimMatrix: 
                fullSimMatrix[aid1] = {0: 0.0 for _ in range(0)}
            if aid2 not in fullSimMatrix[aid1]:
                fullSimMatrix[aid1][aid2] = 0.0
            fullSimMatrix[aid1][aid2] += score


#### Section C: Train the similarity matrices
1. Derive the total like score first
2. Train 2 similarity matrices, one using iuf(Inverse User Frequence), the other using time_decay method. 

In [6]:
%%time
## get the Total Like matrix
item_total_likes = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.float64)

getItemTotalLikesNaive(aids, ops, item_total_likes, ACTION_WEIGHTS)

CPU times: user 18.8 s, sys: 763 ms, total: 19.6 s
Wall time: 19.9 s


In [7]:
%%time
simMatrices = {}   ## store a few different similarity matrices using different scoring system, for different prediction type
TRIM_CYCLES = 1000   ## trim full sim matrix every XX batches. 
MODES_TO_TRAIN = ["iuf", "time_decay"]

for mode in MODES_TO_TRAIN:
    ## the nested dict to store full sim matrix, {itemX: {itemY: score, itemZ: score, ...}}
    fullSimMatrix = nb.typed.Dict.empty(
            key_type = nb.types.int64,
            value_type = nb.typeof(nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)))
    max_idx = len(df)
    batch_idx = 1  ## compute sim matrix for PARALLEL # of rows per batch, have a total of max_idx/PARALLEL batches.
    for idx in tqdm(range(0, max_idx, PARALLEL)):
        rows = df.iloc[idx: min(idx + PARALLEL, max_idx)][['session', 'start_idx', 'total_action', 'session_start_time']].values
        getSimScoreBatch(aids, ts, ops, rows, fullSimMatrix, ACTION_WEIGHTS, item_total_likes, mode=mode)
        batch_idx += 1
        if batch_idx % TRIM_CYCLES == 0:
            print("batch_idx: ", batch_idx)
            trim_simMatrix_topk(fullSimMatrix, 100)
            gc.collect()
            # break

    
    ## trim top 50 when the training is complete
    trim_simMatrix_topk(fullSimMatrix, 80)   ## TODO: make this num small enough to reduce time for normalization
    ## max norm of each score
    maxNormSimMatrix(fullSimMatrix)
    
    simMatrices[mode] = fullSimMatrix
    
    del fullSimMatrix
    gc.collect()

  8%|▊         | 998/12079 [01:35<23:50,  7.75it/s]

batch_idx:  1000


 17%|█▋        | 1999/12079 [05:02<31:26:53, 11.23s/it]

batch_idx:  2000


 25%|██▍       | 2999/12079 [07:39<15:45:28,  6.25s/it]

...

batch_idx:  11000


 99%|█████████▉| 11999/12079 [28:57<09:25,  7.07s/it]  

batch_idx:  12000


100%|██████████| 12079/12079 [28:59<00:00,  6.94it/s]


CPU times: user 1h 36min 27s, sys: 53min 3s, total: 2h 29min 31s
Wall time: 56min 1s


In [9]:
## A sanity check
#simMatrices["iuf"][1517085]

In [20]:

len(simMatrices["iuf"])

1814440

In [22]:
gc.collect()

11016

## 2. Inference -- Make prediction using the matrices derived from above. 

#### Section D: Utils for inference:
1. Select top items to recommend in re-ranking
2. Compute Real time importance of each action (Not in use currently).

In [8]:
@nb.jit(nopython = True)
def heap_topk_return_list(item_cnt_dict, cap):
    """
    get the top cap(k) elements of the cnt dict based on value, using a min-heap structure, return a list with top "cap" elements with highest score
    """
    q = [(np.float64(0), np.int64(0)) for _ in range(0)]  ## generate empty queue to implement a heap, 
    for item_ref, sim_score in item_cnt_dict.items():   ## read in the dict in heap structure
        heapq.heappush(q, (sim_score, item_ref))   ## push the <sim_score, item_ref_id> pair into min-heap, using sim_score for order
        if len(q) > cap:
            heapq.heappop(q)
            
    res = [heapq.heappop(q)[1] for _ in range(len(q))][::-1]
    
    return res

#### Section E: Main Logic in Making Inferences
1. clicks_inferences: time_decay sim matrix + regular action weights <1, 6, 3>.
2. carts_inferencs: iuf sim matrix + weights <4, 2, 5> (as clicks actions tend to lead to cart action next).
3. orders_inferences: iuf sim matrix + regular action weights <1, 6, 3>.

In [9]:
@nb.jit(nopython=True)
def inference_single_session(session, starting_idx, length, start_time, aids, ops, ts, result, full_sim_matrix, test_ops_weights):
    PREV_INTERACT_BONUS = 10
    NEARBY_ACTION_BONUS = 1.5
    
    ending_idx = starting_idx + length
    end_time = ts[ending_idx]
    
    candidates = aids[starting_idx: ending_idx][::-1]
    candidates_ops = ops[starting_idx: ending_idx][::-1]
    
    ## record all potential aid that might be relevant
    potential_to_recommend = nb.typed.Dict.empty(key_type=nb.types.int64, value_type=nb.types.float64)
    
    ## get unique aid of each session 
    unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)
    for a in candidates:
        unique_aids[a] = 0
    
    ## Sequence weight to all the candidates, from near to far 
    sequence_weight = np.power(2, np.linspace(0.3, 1, len(candidates)))[::-1] - 1
    
    ## Time weight of all candidates, from near to far
    time_weights = []
    for idx in range(starting_idx, ending_idx):
        if end_time - ts[idx] < 2 * 60 * 60:   ## apply nearby action bonus
            time_weight = (1 + 0.5 ** ((end_time - ts[idx])/(end_time - start_time))) * NEARBY_ACTION_BONUS
        else:
            time_weight = 1 + 0.5 ** ((end_time - ts[idx])/(end_time - start_time))
        time_weights.append(time_weight)
    time_weights = time_weights[::-1]
    
    
    ## making inference
    if len(unique_aids) >= 20:  
        for aid, op, seq_w, time_w in zip(candidates, candidates_ops, sequence_weight, time_weights):
            if aid not in potential_to_recommend:
                potential_to_recommend[aid] = 0
            potential_to_recommend[aid] += seq_w * time_w * test_ops_weights[op] #* PREV_INTERACT_BONUS
    else:   ## otherwise, fill the rest with similar items.
        for aid, op, seq_w, time_w in zip(candidates, candidates_ops, sequence_weight, time_weights):
            if aid not in potential_to_recommend:
                potential_to_recommend[aid] = 0
            potential_to_recommend[aid] += np.inf #seq_w * time_w * test_ops_weights[op] * PREV_INTERACT_BONUS
            ## adding the similar items, if full_sim_matrix don't have such record, skip. 
            if aid not in full_sim_matrix:
                continue
            for similar_item in full_sim_matrix[aid]:
                ## if sim_item is in candidates, would be included above anyways, skip 
                if similar_item in candidates:
                    continue
                if similar_item not in potential_to_recommend:
                    potential_to_recommend[similar_item] = 0
                potential_to_recommend[similar_item] += seq_w * time_w * test_ops_weights[op] * full_sim_matrix[aid][similar_item]  ## no PREV_INTERACT_BONUS as expected, replaced with sim_matrix scores
    result[session] = np.array(heap_topk_return_list(potential_to_recommend, 20))
    
@nb.jit(nopython=True)
def run_inference_parallel(rows, aids, ops, ts, result, full_sim_matrix, test_ops_weights):
    for row_idx in nb.prange(len(rows)):
        session, starting_idx, length, start_time = rows[row_idx]
        inference_single_session(session, starting_idx, length, start_time, aids, ops, ts, result, full_sim_matrix, test_ops_weights)

In [80]:
%%time
result_iuf = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])

result_iuf_2 = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])

result_time_decay = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])

for row_idx in tqdm(range(len(df) - len(df_test), len(df), PARALLEL)):
    start_row = row_idx
    end_row = min(row_idx + PARALLEL, len(df))
    rows = df.iloc[start_row: end_row][['session', 'start_idx', 'total_action', 'session_start_time']].values
    run_inference_parallel(rows, aids, ops, ts, result_iuf, simMatrices["iuf"], np.array([2.0, 6.0, 6.0]))
    run_inference_parallel(rows, aids, ops, ts, result_iuf_2, simMatrices["iuf"], np.array([4.0, 2.0, 5.0]))   ## considebly add the weights for click action in the real time.
    run_inference_parallel(rows, aids, ops, ts, result_time_decay, simMatrices["time_decay"], np.array([3.0, 6.0, 3.0]))

100%|██████████| 1742/1742 [08:22<00:00,  3.47it/s]

CPU times: user 6min 34s, sys: 53.8 s, total: 7min 28s
Wall time: 8min 23s


### Submissions - Convert results to csv, get validation set result

In [83]:
%%time
subs = []
op_names = ["clicks", "carts", "orders"]

for result, op in zip([result_time_decay, result_iuf_2, result_iuf], op_names):
    sub = pd.DataFrame({"session_type": result.keys(), "labels": result.values()})
    sub.session_type = sub.session_type.astype(str) + f"_{op}"
    sub.labels = sub.labels.apply(lambda x: " ".join(x.astype(str)))
    subs.append(sub)
    
submission = pd.concat(subs).reset_index(drop=True)
#sub.sort_values(by=["session_type"])  ## optional
#submission.to_csv('submission.csv', index = False)
submission

CPU times: user 5min 33s, sys: 18.9 s, total: 5min 52s
Wall time: 6min 12s


,session_type,labels
0,11098528_clicks,11830 1732105 588923 571762 884502 876129 1157...
1,11098529_clicks,1105029 1632356 1049489 612829 295362 333991 1...
2,11098530_clicks,409236 264500 1603001 583026 963957 254154 752...
3,11098531_clicks,1728212 1557766 1553691 1449555 1365569 130963...
4,11098532_clicks,876469 7651 1202618 1159379 77906 476681 17040...
...,...,...
5351206,12899774_orders,33035 1539309 819288 771913 31490 95488 218795...
5351207,12899775_orders,1743151 1760714 1163166 1255910 1498443 783827...
5351208,12899776_orders,548599 695829 1737908 773354 1762353 1440959 9...
5351209,12899777_orders,384045 1308634 1688215 395762 703474 1486067 2...


In [85]:
%%time
submission.to_csv('../../allData/validationData/p_v_579_80_items.csv', index = False)

CPU times: user 47 s, sys: 3.32 s, total: 50.3 s
Wall time: 50.9 s


### Section F: Saving Feature, Run E or F, depends on mode, this will not only generate result, but the featues associated.

##### Utils for features save

In [10]:
@nb.jit(nopython=True)
def update_feature_vec(aid, features_tuple_arr, features_idx_map, new_feat_tuple):
    ## append features
    if aid not in features_idx_map:
        features_tuple_arr.append(new_feat_tuple)
        new_pos = len(features_tuple_arr)-1
        ## save the position in the tuple arr
        features_idx_map[aid] = new_pos
    else: # <is_prev_int, seq_w, time_w, associated_action, session_len,.. >
        features_tuple_arr[features_idx_map[aid]] = (new_feat_tuple[0], 
                                                     features_tuple_arr[features_idx_map[aid]][1] + new_feat_tuple[1], 
                                                     features_tuple_arr[features_idx_map[aid]][2] + new_feat_tuple[2], 
                                                     features_tuple_arr[features_idx_map[aid]][3] + new_feat_tuple[3],
                                                     new_feat_tuple[4],
                                                     new_feat_tuple[5],
                                                     new_feat_tuple[6])

##### Main feature save logics

In [17]:
@nb.jit(nopython=True)
def save_feature_single_session(session, starting_idx, length, start_time, aids, ops, ts, result, full_sim_matrix, test_ops_weights):
    PREV_INTERACT_BONUS = 10
    NEARBY_ACTION_BONUS = 1.5
    
    ending_idx = starting_idx + length 
    end_time = ts[ending_idx - 1]
    
    candidates = aids[starting_idx: ending_idx][::-1]
    candidates_ops = ops[starting_idx: ending_idx][::-1]
    
    ## record all potential aid that might be relevant
    potential_to_recommend = nb.typed.Dict.empty(key_type=nb.types.int64, value_type=nb.types.float64)
    
    ## get unique aid of each session 
    unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)
    for a in candidates:
        unique_aids[a] = 0
    
    ## Sequence weight to all the candidates, from near to far 
    sequence_weight = np.power(2, np.linspace(0.3, 1, len(candidates)))[::-1] - 1
    
    ## Time weight of all candidates, from near to far
    time_weights = []
    time_lapse = end_time - start_time + 1  ## +1 to avoid zero
    for idx in range(starting_idx, ending_idx):
        if end_time - ts[idx] < 2 * 60 * 60:   ## apply nearby action bonus
            time_weight = (1 + 0.5 ** ((end_time - ts[idx])/time_lapse)) * NEARBY_ACTION_BONUS
        else:
            time_weight = 1 + 0.5 ** ((end_time - ts[idx])/time_lapse)
        time_weights.append(time_weight)
    time_weights = time_weights[::-1]
    
    ## feature vector template: [aid: <is_prev_int, seq_w, time_w, associated_action, session_len,.. >]
    features_tuple_arr = nb.typed.List()
    features_tuple_arr.append((bool(0), np.float64(0.0), np.float64(0.0), np.int64(0), np.int64(0), np.int64(0), np.float64(0.0)))
    features_idx_map = nb.typed.Dict.empty(key_type=nb.types.int64, value_type=nb.types.int64)
    
    ## making inference
    if len(unique_aids) >= 20:  
        for aid, op, seq_w, time_w in zip(candidates, candidates_ops, sequence_weight, time_weights):
            if aid not in potential_to_recommend:
                potential_to_recommend[aid] = 0
            ## caculate scores
            potential_to_recommend[aid] += seq_w * time_w * test_ops_weights[op] #* PREV_INTERACT_BONUS
            ## append features
            update_feature_vec(aid, features_tuple_arr, features_idx_map, (1, seq_w, time_w, op, length, len(unique_aids), potential_to_recommend[aid]))
    else:   ## otherwise, fill the rest with similar items.
        for aid, op, seq_w, time_w in zip(candidates, candidates_ops, sequence_weight, time_weights):
            if aid not in potential_to_recommend:
                potential_to_recommend[aid] = 0
            ## get the scores
            potential_to_recommend[aid] += seq_w * time_w * test_ops_weights[op] * PREV_INTERACT_BONUS
            ## append features
            update_feature_vec(aid, features_tuple_arr, features_idx_map, (1, seq_w, time_w, op, length, len(unique_aids), potential_to_recommend[aid]))
            ## adding the similar items, if full_sim_matrix don't have such record, skip. 
            if aid not in full_sim_matrix:
                continue
            for similar_item in full_sim_matrix[aid]:
                ## if sim_item is in candidates, would be included above anyways, skip 
                if similar_item in candidates:
                    continue
                if similar_item not in potential_to_recommend:
                    potential_to_recommend[similar_item] = 0
                potential_to_recommend[similar_item] += seq_w * time_w * test_ops_weights[op] * full_sim_matrix[aid][similar_item]  ## no PREV_INTERACT_BONUS as expected, replaced with sim_matrix scores
                ## append features
                update_feature_vec(similar_item, features_tuple_arr, features_idx_map, (0, seq_w, time_w, op, length, len(unique_aids), potential_to_recommend[similar_item]))

    result[session] = np.array(heap_topk_return_list(potential_to_recommend, 100))  ## Take top 100 for validation runs. 
    
    feature_tuples_this_session = []
    for aid in result[session]:
#         features_save[(session, aid)] = features_tuple_arr[features_idx_map[aid]]
        feature_tuples_this_session.append(features_tuple_arr[features_idx_map[aid]])
    
    return feature_tuples_this_session

In [18]:
%%time
result_iuf_orders = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])

# result_iuf_carts = nb.typed.Dict.empty(
#     key_type = nb.types.int64,
#     value_type = nb.types.int64[:])

# result_time_decay_clicks = nb.typed.Dict.empty(
#     key_type = nb.types.int64,
#     value_type = nb.types.int64[:])

features_all_sessions = []

for row_idx in tqdm(range(len(df) - len(df_test), len(df), PARALLEL)):
    start_row = row_idx
    end_row = min(row_idx + PARALLEL, len(df))
    rows = df.iloc[start_row: end_row][['session', 'start_idx', 'total_action', 'session_start_time']].values
#     save_features_parallel(rows, aids, ops, ts, result_iuf_orders, simMatrices["iuf"], np.array([2.0, 8.0, 6.0]), orders_features_save)  
    ## run things in parallel
    for row_idx in nb.prange(len(rows)):
        session, starting_idx, length, start_time = rows[row_idx]
        features_tuples_this_session = save_feature_single_session(session, starting_idx, length, start_time, aids, ops, ts, result_iuf_orders, simMatrices["iuf"], np.array([2.0, 6.0, 6.0]))
        features_all_sessions.append(features_tuples_this_session)
#     gc.collect()

100%|██████████| 1742/1742 [14:33<00:00,  1.99it/s]  

CPU times: user 4min 21s, sys: 2min 25s, total: 6min 47s
Wall time: 14min 37s


In [20]:
gc.collect()

KeyboardInterrupt: 

In [21]:
%%time
features_save = pd.DataFrame({"session": result_iuf_orders.keys(), "aids": result_iuf_orders.values(), "feature_tuple": features_all_sessions})
features_save = features_save.set_index(['session']).apply(pd.Series.explode).reset_index()
del features_all_sessions
gc.collect()

CPU times: user 33.1 s, sys: 1min 49s, total: 2min 22s
Wall time: 8min 42s


8032

In [22]:
features_save["prev_int"] = features_save["feature_tuple"].apply(lambda x: x[0])
features_save["seq_w"] = features_save["feature_tuple"].apply(lambda x: x[1])
features_save["time_w"] = features_save["feature_tuple"].apply(lambda x: x[2])
features_save["ops_total"] = features_save["feature_tuple"].apply(lambda x: x[3])
features_save["session_len"] = features_save["feature_tuple"].apply(lambda x: x[4])
features_save["session_unique_aid"] = features_save["feature_tuple"].apply(lambda x: x[5])
features_save["rank_score"] = features_save["feature_tuple"].apply(lambda x: x[6])
features_save = features_save.drop("feature_tuple", axis = 1)
gc.collect()

0

In [23]:
features_save

,session,aids,prev_int,seq_w,time_w,ops_total,session_len,session_unique_aid,rank_score
0,11098528,11830,True,0.231144,3.0,0,1,1,13.868665
1,11098528,1732105,False,0.231144,3.0,0,1,1,1.386866
2,11098528,588923,False,0.231144,3.0,0,1,1,0.733088
3,11098528,571762,False,0.231144,3.0,0,1,1,0.376020
4,11098528,884502,False,0.231144,3.0,0,1,1,0.354726
...,...,...,...,...,...,...,...,...,...
156190584,12899778,1745743,False,0.231144,3.0,0,1,1,0.052593
156190585,12899778,1028447,False,0.231144,3.0,0,1,1,0.052512
156190586,12899778,1543085,False,0.231144,3.0,0,1,1,0.052005
156190587,12899778,1653733,False,0.231144,3.0,0,1,1,0.051983


In [24]:
features_save.to_parquet("../../allData/features/orders_features_100_per_session.parquet")

In [16]:
gc.collect()

20